# Preprocessing Only Workflow

In this workflow, CFRL takes in an offline trajectory and then preprocesses the offline trajectory 
using `SyntheticPreprocessor`. The final output of the workflow is the preprocessed (debiased) 
offline trajectory. This workflow is appropriate when the user does not want to train policies using 
CFRL. Instead, the user can take the preprocessed trajectory to train a counterfactually fair policy 
using another reinforcement learning library or application that better fits their needs.

We begin by importing the liberaries needed for this demonstration.

In [7]:
# Need this temporarily to import CFRL before it is officially published to PyPI
import sys
sys.path.append("E:/learning/university/MiSIL/CFRL Python Package/CFRL")

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from CFRL.reader import read_trajectory_from_dataframe, convert_trajectory_to_dataframe
from CFRL.preprocessor import SequentialPreprocessor

## Data Loading

In this demonstration, we use an offline trajectory generated from a `SyntheticEnvironment` using some pre-specified transition rules. Although it is actually synthesized, we treat it as if it is from some unknown environment for pedagogical convenience in this demonstration.

The trajectory contains 100 individuals (i.e. $N=100$) and 10 transitions (i.e. $T=10$). The actions are binary (0 or 1) and were sampled using a random policy that selects 0 or 1 randomly with equal probability. It is stored in a tabular format in a `.csv` file. The sensitive attribute variable is bivariate, stored in columns `z1` and `z2`. The legit values of the sensitive attribute are $[0, 0]$, $[1, 0]$, $[0, 1]$, and $[1, 1]$. The state variable is tri-variate, stored in columns `state1`, `state2`, and `state3`. The actions are stored in the column `action` and rewards in the column `reward`. The tabular data also includes an extra inrrelevant column `timestamp`. 

We can load and view the tabular data.

In [14]:
trajectory = pd.read_csv('../data/sample_data.csv')
trajectory

,Unnamed: 0,ID,timestamp,z1,z2,action,reward,state1,state2,state3
0,0,1.0,1.0,0.0,1.0,NaN,NaN,3.124345,0.888244,0.971828
1,1,1.0,2.0,0.0,1.0,1.0,7.337698,1.996327,4.061797,1.776387
2,2,1.0,3.0,0.0,1.0,0.0,8.859261,2.659805,3.083692,3.285352
3,3,1.0,4.0,0.0,1.0,0.0,9.846707,3.482482,3.038571,1.886212
4,4,1.0,5.0,0.0,1.0,0.0,8.211237,3.918101,1.915400,2.059517
...,...,...,...,...,...,...,...,...,...,...
1095,1095,100.0,7.0,1.0,1.0,1.0,25.666879,8.994854,9.426115,7.743749
1096,1096,100.0,8.0,1.0,1.0,0.0,27.661526,7.707395,8.509083,10.620464
1097,1097,100.0,9.0,1.0,1.0,0.0,28.743075,8.344374,8.768804,9.468524
1098,1098,100.0,10.0,1.0,1.0,1.0,29.696987,8.098696,9.995528,10.065131


We now read the trajectory from the tabular format into Trajectory Arrays.

In [18]:
zs, states, actions, rewards, ids = read_trajectory_from_dataframe(
                                                data=trajectory, 
                                                z_labels=['z1', 'z2'], 
                                                state_labels=['state1', 'state2', 'state3'], 
                                                action_label='action', 
                                                reward_label='reward', 
                                                id_label='ID', 
                                                T=10
                                                )

## Preprocessor Training

Before preprocessing the trajectory, we need to first train a preprocessor. To mitigate overfitting, we use a random subset of 50 individuals in the trajectory to train the preprocessor. The remaining 50 individuals will be actually preprocessed. We now form these two sets.

In [20]:
(
    zs_train, zs_prepro, 
    states_train, states_prepro, 
    actions_train, actions_prepro, 
    rewards_train, rewards_prepro, 
    ids_train, ids_prepro
) = train_test_split(zs, states, actions, rewards, ids, test_size=0.5)

We now use the training set to train a `SequentialPreprocessor`.

In [ ]:
sp = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                            num_actions=2, 
                            cross_folds=1, 
                            mode='single', 
                            reg_model='nn')
sp.train_preprocessor(zs=zs_train, xs=states_train, actions=actions_train, rewards=rewards_train)

100%|██████████| 1000/1000 [00:04<00:00, 203.51it/s]


(array([[[-0.72528759,  0.7386152 ,  0.51817813, ...,  1.38352998,
           2.5809271 ,  2.31342101],
         [ 1.68809123, -1.99483037, -1.40597664, ...,  4.92257044,
           2.2346592 ,  2.29302509],
         [-0.35345339, -1.50415468, -1.17551192, ...,  5.06302697,
           2.97395653,  4.74342934],
         ...,
         [-3.82499722, -5.7901147 , -5.09233495, ...,  8.63004514,
           6.57981509,  6.80623821],
         [-5.37691526, -2.7063859 , -4.89311208, ...,  8.74704381,
           9.95186014,  7.73383056],
         [-5.25975988, -4.47628941, -3.90354107, ...,  8.58067086,
           8.93510757,  8.30239012]],
 
        [[ 1.29266932,  3.18601381,  0.45612234, ...,  3.40148689,
           5.02832571,  2.25136522],
         [ 2.3535193 , -0.1892715 ,  1.63780025, ...,  5.74298809,
           3.86517102,  5.63737658],
         [ 0.73956062, -1.75676155,  1.00539288, ...,  7.21622463,
           3.55873608,  6.88828572],
         ...,
         [-2.16910469, -1.2925976

## Data Preprocessing

We now preprocess the remaining data that are not in the training set.

In [23]:
states_tilde, rewards_tilde = sp.preprocess_multiple_steps(zs=zs_prepro, 
                                                           xs=states_prepro, 
                                                           actions=actions_prepro, 
                                                           rewards=rewards_prepro)

## Data Exporting

Finally, we convert the preprocessed trajectory back into the tabular format so that it is easier to store and manage. For simplicity, we call the new state as `state1`, ..., `state12`, which is the default option provided by `convert_trajectory_to_dataframe` (so we do not need to specify the `state_labels` argument).

In [26]:
preprocessed_trajectory = convert_trajectory_to_dataframe(
                                        zs=zs_prepro, 
                                        states=states_tilde, 
                                        actions=actions_prepro, 
                                        rewards=rewards_tilde, 
                                        ids=ids_prepro, 
                                        z_labels=['z1', 'z2'], 
                                        action_label='action', 
                                        reward_label='reward', 
                                        id_label='ID', 
                                        T_label='time_step'
                                        )
preprocessed_trajectory

,ID,time_step,z1,z2,action,reward,state1,state2,state3,state4,state5,state6,state7,state8,state9,state10,state11,state12
0,3.0,1.0,1.0,0.0,NaN,NaN,2.291870,-0.388533,0.536031,3.876724,0.022480,1.538154,3.244812,0.738793,1.819039,4.400687,1.453779,2.331274
1,3.0,2.0,1.0,0.0,0.0,6.977079,0.546537,-0.888551,-0.433003,2.340911,0.520398,1.552233,2.691916,1.545872,2.756207,4.128972,2.882218,3.592735
2,3.0,3.0,1.0,0.0,1.0,5.986007,-0.317321,0.426840,-0.239190,3.076584,2.640750,2.175063,3.806165,3.190534,3.246579,5.551201,5.951027,5.302299
3,3.0,4.0,1.0,0.0,0.0,8.349300,-1.558086,-0.211909,-2.704830,1.568007,3.131790,0.328390,2.683652,3.694238,2.035955,4.930791,6.958170,4.831392
4,3.0,5.0,1.0,0.0,0.0,6.803303,-2.286527,-1.657626,-3.048321,2.264041,2.128014,0.262472,2.922773,2.691943,2.060497,6.089287,6.157106,4.888200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,65.0,7.0,0.0,0.0,1.0,11.454567,-1.950252,0.395168,0.251163,3.508967,6.045513,5.937161,4.292503,6.236781,6.867482,8.958252,11.051716,11.652609
546,65.0,8.0,0.0,0.0,1.0,14.471473,-2.123382,-0.791285,-1.603140,3.975209,5.411002,3.415106,4.559382,5.816836,4.142472,10.004624,10.910431,9.038021
547,65.0,9.0,0.0,0.0,0.0,12.823564,-1.407231,-2.700433,-3.225772,5.014303,3.303796,2.860268,5.784858,3.617949,3.482676,10.271727,9.204033,8.670113
548,65.0,10.0,0.0,0.0,1.0,11.335298,-2.312682,-3.734385,-2.394897,4.569107,2.844652,4.259174,4.482447,2.639961,4.916410,10.477112,8.344645,10.722385


## Alternative: Preprocessing All Individuals

Sometimes, the number of individuals in the trajectory is small. In this case, if we only preprocess a subset of individuals, the resulting preprocessed trajectory might be too small to be useful. In this case, we can directly preprocess all individuals using the `train_preprocessor()` function when we have a relatively large number of `cross_folds`.

When `cross_folds=K` where `K` is greater than 1, `train_preprocessor()` will internally divide the training data into `K` folds. For each $i=1,\dots,k$, it trains a model based on all the folds other than the $i$-th one, which is then used to preprocess data in the $i$-th fold. This results in `K` folds of preprocessed data, each of which is processed using a model that is trained on the other folds. These `K` folds of data are then combined and returned by `train_preprocessor()`. This method allows us to preprocess all individuals in the trajectory while reducing overfitting.

To use this functionality, we first initialize a `SequentialPreprocessor` with `cross_folds` greater than 1. We use `cross_folds=5` here.

In [27]:
sp_cf5 = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                                num_actions=2, 
                                cross_folds=5, 
                                mode='single', 
                                reg_model='nn')

We now simultaneously train the preprocessor and preprocess all individuals in the trajectory using the precedure described above.

In [29]:
states_tilde_cf5, rewards_tilde_cf5 = sp_cf5.train_preprocessor(zs=zs, 
                                                                xs=states, 
                                                                actions=actions, 
                                                                rewards=rewards)

100%|██████████| 1000/1000 [00:09<00:00, 108.90it/s]


We can now convert the preprocessed trajectory into the tabular format.

In [30]:
preprocessed_trajectory_cf5 = convert_trajectory_to_dataframe(
                                            zs=zs_prepro, 
                                            states=states_tilde, 
                                            actions=actions_prepro, 
                                            rewards=rewards_tilde, 
                                            ids=ids_prepro, 
                                            z_labels=['z1', 'z2'], 
                                            action_label='action', 
                                            reward_label='reward', 
                                            id_label='ID', 
                                            T_label='time_step'
                                            )
preprocessed_trajectory_cf5

,ID,time_step,z1,z2,action,reward,state1,state2,state3,state4,state5,state6,state7,state8,state9,state10,state11,state12
0,3.0,1.0,1.0,0.0,NaN,NaN,2.291870,-0.388533,0.536031,3.876724,0.022480,1.538154,3.244812,0.738793,1.819039,4.400687,1.453779,2.331274
1,3.0,2.0,1.0,0.0,0.0,6.977079,0.546537,-0.888551,-0.433003,2.340911,0.520398,1.552233,2.691916,1.545872,2.756207,4.128972,2.882218,3.592735
2,3.0,3.0,1.0,0.0,1.0,5.986007,-0.317321,0.426840,-0.239190,3.076584,2.640750,2.175063,3.806165,3.190534,3.246579,5.551201,5.951027,5.302299
3,3.0,4.0,1.0,0.0,0.0,8.349300,-1.558086,-0.211909,-2.704830,1.568007,3.131790,0.328390,2.683652,3.694238,2.035955,4.930791,6.958170,4.831392
4,3.0,5.0,1.0,0.0,0.0,6.803303,-2.286527,-1.657626,-3.048321,2.264041,2.128014,0.262472,2.922773,2.691943,2.060497,6.089287,6.157106,4.888200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,65.0,7.0,0.0,0.0,1.0,11.454567,-1.950252,0.395168,0.251163,3.508967,6.045513,5.937161,4.292503,6.236781,6.867482,8.958252,11.051716,11.652609
546,65.0,8.0,0.0,0.0,1.0,14.471473,-2.123382,-0.791285,-1.603140,3.975209,5.411002,3.415106,4.559382,5.816836,4.142472,10.004624,10.910431,9.038021
547,65.0,9.0,0.0,0.0,0.0,12.823564,-1.407231,-2.700433,-3.225772,5.014303,3.303796,2.860268,5.784858,3.617949,3.482676,10.271727,9.204033,8.670113
548,65.0,10.0,0.0,0.0,1.0,11.335298,-2.312682,-3.734385,-2.394897,4.569107,2.844652,4.259174,4.482447,2.639961,4.916410,10.477112,8.344645,10.722385
